# Dependency Injection
.NET posee una infraestructura para definir los objetos disponibles a inyectar, para poder implementar inyección de dependencias. 

## Ciclos de vida

- **Transient**: Se creará una instancia cada vez que es solicitada. Cuando es solicitada a través de una request, la instancia es destruida al final de la request. 
- **Scoped**: Se creará una instancia por request (o scope) y será compartida por todos los que la soliciten en la request.  
- **Singleton**: Se creará una instancia la primera vez que es solicitada.

El builder validará al momento de instanciar que no haya dependencias desde un servicio con tiempo de vida mayor a uno con tiempo de vida menor. (Por ejemplo, de un singleton a un scoped) 


## Donde se definen
Las correspondencias se definen a través del WebApplicationBuilder. El mismo define los metodos para agregar servicios con los ciclos de vida soportados


In [ ]:
// definiciones de servicios e interfaces

public interface IUserRepo {
    //....
}

public class UserRepo : IUserRepo {
   //....
}

public interface IAuthService {
    Task Login(string username, string password);
}

public class AuthService : IAuthService {
    private readonly IUserRepo userRepo;

    // se definen las dependencias en un constructor publico (que debe ser unico)
    public AuthService(IUserRepo userRepo) {
        this.userRepo = userRepo;
    }

    public Task Login(string username, string password) {
        //....
    }
}



// codigo de startup

var builder = WebApplicationBuilder.CreateBuilder(args);

// con la clase especifica
builder.Services.AddTransient<AuthService>();

// implementacion de una interfaz
builder.Services.AddTransient<IAuthService, AuthService>();
builder.Services.AddTransient<IUserRepo, UserRepo>();


// otros ciclos de vida
builder.Services.AddSingleton<ClientService>();
builder.Services.AddScoped<ProductService>();

# Autorización
.NET posee dispone de un sistema de autorización basado en **roles** y/o **policies** que pueden definir custom handlers en base a las claims del usuario y el contexto de la request.

## Roles
Un usuario puede tener una coleccion de roles. Por defecto, los mismos se parsean de las claims de rol (http://schemas.microsoft.com/ws/2008/06/identity/claims/role)

## Policies
Se pueden definir policies personalizadas con requerimientos de roles, claims especificas, o requerimientos custom. Para los requerimientos custom se deben implementar un RequirementHandler correspondiente donde se puede acceder al user y el contexto de la request, etc.

## Cómo se configura
A través del metodo de extensión AddAuthorization, y los metodos de extensión de la forma de autenticación que se quiera usar. (Por ejemplo JwtBearer)

In [ ]:
var builder = WebApplicationBuilder.CreateBuilder(args);

// habilitando control de autorización, no se requieren paquetes nuget adicionales
builder.Services.AddAuthorization(options => {
    // aqui se configuran policies basadas custom requirements
    options.ConfigureRequirement<GroupOwnerRequirement>("GroupOwner");

    //ejemplo de policies basadas en claims
    options.AddPolicy("GoogleUsersOnly", policy => policy.RequireClaim("GoogleUserId"));
});

// en caso de configurar custom requirements, se deben declarar los handlers por acá
builder.Services.AddTransient<IAuthorizationHandler, GroupOwnerRequirementHandler>(); // GroupOwnerRequirementHandler : AuthorizationHandler<GroupOwner> (mas detalles adelante)

// habilitando autenticación por JWT, los metodos de extensión son proporcionados por el paquete nuget Microsoft.AspNetCore.Authentication.JwtBearer
// se pueden soportar varios tipos de autenticación (incluso del mismo tipo) lo que permite interactuar con distintos proveedores de autenticacion a la vez
builder.Services.AddAuthentication(options =>
{
    options.DefaultAuthenticateScheme = JwtBearerDefaults.AuthenticationScheme;
    options.DefaultChallengeScheme = JwtBearerDefaults.AuthenticationScheme;
    options.DefaultScheme = JwtBearerDefaults.AuthenticationScheme;
}).AddJwtBearer(options =>
    {
        options.SaveToken = true;
        // options.RequireHttpsMetadata = false;  
        options.TokenValidationParameters = new TokenValidationParameters()
        {
            ValidateIssuer = true,
            ValidateAudience = true,
            ValidAudience = jwtConfiguration.ValidAudience,
            ValidIssuer = jwtConfiguration.ValidIssuer,
            IssuerSigningKey = new SymmetricSecurityKey(Encoding.UTF8.GetBytes(jwtConfiguration.Secret))
        };
    });

## Como se usa
Se debe establecer un atributo ``[Authorize]`` a nivel de controller o metodo del controller para requerir la authenticacion en ese/esos endpoints.
En este atributo se pueden definir los roles que pueden acceder al endpoint y las policies que deben ser cumplidas.

In [ ]:
public class GroupsController : Controller {
    [Authorize(Roles="Admin", Policy = "GroupOwner")]
    public async Task<Group> GetGroup(long groupId) {
        //....
    }
}

## Custom requirements
Para los custom requirements, se deben implementar sus handlers, los cuales deben heredar de ``AuthorizationHandler<TRequirement>`` 

In [ ]:
public class AccessControlRequirement<GroupOwnerRequirement> : AuthorizationHandler<GroupOwnerRequirement> {
    public override async Task HandleRequirementAsync(AuthorizationHandlerContext context, GroupOwnerRequirement requirement)
    {
        // a través del AuthorizationHandlerContext se pueden acceder a distintos datos de la request y el user

        // se pueden obtener claims del usuario
        var userId = context.User.FindFirstValue("userId"); 

        var entityId = default(long?);

        // segun el contexto de autorizacion, se pueden obtener parametros de la url, headers, etc.
        if (context.Resource is DefaultHttpContext httpContext)
        {
            var parameter = httpContext.GetRouteValue("groupId");

            entityId = long.TryParse((string)parameter, out var parsedEntityId)
                ? parsedEntityId
                : throw new CommonException($"groupId parameter not found");
        }

        //buscar relacion en la db

        context.Success();
    }
}

## ASP.NET Identity
A través de los metodos de extensión provistos por Microsoft.AspNetCore.Identity.EntityFrameworkCore, se puede configurar un proveedor de usuarios basado en entity framework. Que incluye:
- Gestión de usuarios y roles
- Generación de tokens

In [ ]:
builder.Services
    .AddIdentityCore<IdentityUser>()
    .AddRoles<IdentityRole>()
    .AddDefaultTokenProviders()
    .AddEntityFrameworkStores<AppDbContext>();

builder.Services.Configure<IdentityOptions>(options =>
{
    // Password settings.
    options.Password.RequireDigit = true;
    options.Password.RequireLowercase = true;
    options.Password.RequireNonAlphanumeric = true;
    options.Password.RequireUppercase = true;
    options.Password.RequiredLength = 8;
    options.Password.RequiredUniqueChars = 1;

    // Lockout settings.
    options.Lockout.DefaultLockoutTimeSpan = TimeSpan.FromMinutes(5);
    options.Lockout.MaxFailedAccessAttempts = 5;
    options.Lockout.AllowedForNewUsers = true;

    // User settings.
    options.User.AllowedUserNameCharacters = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789-._@+";
    options.User.RequireUniqueEmail = false;
});